# Machine Learning Project

## Setup

### Import

In [1]:
# Import os for operating system functions.
import os
# Import numpy for mathematical computation.
import numpy as np
# Import pandas for data manipulation.
import pandas as pd
# Import datetime for local time retrieval.
from datetime import datetime
# Import logging for file logging.
import logging
# Import time for duration measurement.
import time

# Import random, clone, make_scorer, and _fit_and_score for simulated annealing.
import random
from sklearn.base import clone
from sklearn.metrics import make_scorer
from sklearn.model_selection._validation import _fit_and_score

# Import StandardScaler for data pre-processing.
from sklearn.preprocessing import StandardScaler
# Import train_test_split for model selection.
from sklearn.model_selection import train_test_split
# Import KFold and f1_score for cross-validation.
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, classification_report

# Import SK, XGB, and CatBoost classifiers.
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

### Initialisation

In [2]:
logging.basicConfig(
	filename="main.log",
	level=logging.INFO,
	format="%(asctime)s - %(message)s",
	datefmt="%Y-%m-%d %H:%M:%S"
)

PARAMETER_MAPPINGS = {
	"α": "learning_rate",
	"τ": "max_iter",
	"θ": "max_leaf_nodes",
	"Δ": "max_depth",
	"l": "min_samples_leaf",
	"seed": "random_state"
}

MODEL_CLASSES = {
	"SKLknn": KNeighborsClassifier,
	"SKLsvm": SVC,
	"SKLrf": RandomForestClassifier,
	"SKLgb": GradientBoostingClassifier,
	"SKLhgb": HistGradientBoostingClassifier,
	"XGBgb": XGBClassifier,
	"CBgb": CatBoostClassifier
}

# Load train and test datasets.
S_train = pd.read_csv("./data/train.csv")
S_train_tfidf = pd.read_csv("./data/train_tfidf_features.csv")
S_test = pd.read_csv("./data/test.csv")
S_test_tfidf = pd.read_csv("./data/test_tfidf_features.csv")

# Extract train features, train labels, and test features.
X_train = S_train_tfidf.iloc[:, 2:].values
y_train = S_train["label"].values.reshape(-1, 1)
X_test = S_test_tfidf.iloc[:, 1:].values

## Task 1

### logreg Model

In [3]:
def σ(z): return 1 / (1 + np.exp(-z))
def bce_loss(y, ŷ): return (-1/(y.shape[0])) * np.sum(y * np.log(ŷ) + (1 - y) * np.log(1 - ŷ))

# Return dw and db, for some X, y, ŷ, w, R, and λ.
def gradients_logreg(X, y, ŷ, w, R=None, λ=0):
	m, _ = X.shape
	dw = 1/m * np.dot(X.T, (ŷ - y))
	db = 1/m * np.sum(ŷ - y)
	if R == "L2":
		dw += λ * w / m
	elif R == "L1":
		dw += λ * np.sign(w) / m
	return dw, db

# Return (w, b) from gradient descent on X_train and y_train, for some τ, α, G, β, R, and λ.
def train_logreg(X_train, y_train, τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	m, n = X_train.shape
	w, b = np.zeros((n, 1)), 0
	for epoch in range(τ):
		if G == "full-batch":
			X_batch, y_batch = X_train, y_train
			ŷ = σ(np.dot(X_batch, w) + b)
			dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
			w, b = w - α*dw, b - α*db
		elif G == "mini-batch":
			for i in range(0, m, β):
				X_batch, y_batch = X_train[i:i+β], y_train[i:i+β]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
		elif G == "stochastic":
			for i in range(m):
				X_batch, y_batch = X_train[i:i+1], y_train[i:i+1]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
	return w, b

# Return array of predictions, where each prediction is 1 if corresponding ŷ entry > 0.5, and 0 otherwise.
def predict_logreg(wb_tuple, X):
	w, b = wb_tuple
	ŷ = σ(np.dot(X, w) + b)
	return np.array([1 if p > 0.5 else 0 for p in ŷ])

# Train model, make predictions, and save predictions to CSV file.
def generate_predictions_logreg(τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	start_time = time.time()
	w, b = train_logreg(np.array(X_train), np.array(y_train), τ, α, G, β, R, λ)
	predictions = predict_logreg((w, b), np.array(X_test))
	os.makedirs("./predictions/logreg/", exist_ok=True)
	file_name = os.path.join("./predictions/logreg/", f"τ={τ},α={α},G={G},β={β},R={R},λ={λ}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

## Task 2

### PCA

In [4]:
def apply_pca(x):
	scaler = StandardScaler()
	X_train_scaled = scaler.fit_transform(X_train)
	X_test_scaled = scaler.transform(X_test)
	if 0 <= x <= 1:
		# x represents variance threshold.
		pca = PCA(n_components=None)
		pca.fit(X_train_scaled)
		c = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= x) + 1
		v = x
	else:
		# x represents number of components.
		pca = PCA(n_components=x)
		pca.fit(X_train_scaled)
		c = x
		v = sum(pca.explained_variance_ratio_)
	# Transform train and test datasets.
	X_train_pca = pca.transform(X_train_scaled)
	X_test_pca = pca.transform(X_test_scaled)
	return X_train_pca, X_test_pca, c, v

### SKLknn Model

In [5]:
def train_SKLknn(X_train, y_train, k=5, W="uniform", p=2, m="minkowski"):
	model = KNeighborsClassifier(
		n_neighbors=k,
		weights=W,
		p=p,
		metric=m,
		n_jobs=-1
	)
	model.fit(X_train, y_train)
	return model

def predict_SKLknn(model, X): return model.predict(X)

def generate_predictions_SKLknn(k=5, W="uniform", p=2, m="minkowski"):
	start_time = time.time()
	model = train_SKLknn(np.array(X_train), np.array(y_train), k, W, p, m)
	predictions = predict_SKLknn(model, np.array(X_test))
	os.makedirs("./predictions/SKLknn/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLknn/", f"k={k},W={W},p={p},m={m}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### Combination

In [6]:
# Train model, make model predictions, and save model predictions to CSV file.
def generate_predictions_pcaknn(x):
	start_time = time.time()
	X_train_pca, X_test_pca, c, v = apply_pca(x)
	model = train_SKLknn(X_train_pca, y_train, k=2)
	predictions = predict_SKLknn(model, X_test_pca)
	os.makedirs("./predictions/pcaknn/", exist_ok=True)
	file_name = os.path.join("./predictions/pcaknn/", f"pcaknn(ρ={c},ν={v:.2f}).csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

## Task 3: Other Models

### SKLlogreg Model (WIP)

### SKLrf Model

In [7]:
def train_SKLrf(X_train, y_train, η=100, C="gini", Δ=None, ψ=2, l=1, θ=None, seed=None):
	model = RandomForestClassifier(
		n_estimators=η,
		criterion=C,
		max_depth=Δ,
		min_samples_split=ψ,
		min_samples_leaf=l,
		max_leaf_nodes=θ,
		n_jobs=-1,
		random_state=seed
	)
	model.fit(X_train, y_train.ravel())
	return model

def predict_SKLrf(model, X): return model.predict(X)

def generate_predictions_SKLrf(η=100, C="gini", Δ=None, ψ=2, l=1, θ=None, seed=None):
	start_time = time.time()
	model = train_SKLrf(np.array(X_train), np.array(y_train), η, C, Δ, ψ, l, θ, seed)
	predictions = predict_SKLrf(model, np.array(X_test))
	os.makedirs("./predictions/SKLrf/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLrf/", f"η={η},C={C},Δ={Δ},ψ={ψ},l={l},θ={θ},seed={seed}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### SKLsvm Model

In [8]:
# Train model, make predictions, and save predictions to CSV file.
def train_SKLsvm(X_train, y_train, λ=1.0, K="rbf", d=3, γ="scale", τ=-1, seed=None):
	model = SVC(
		C=λ,
		kernel=K,
		degree=d,
		gamma=γ,
		max_iter=τ,
		random_state=seed
	)
	model.fit(X_train, y_train.ravel())
	return model

def predict_SKLsvm(model, X): return model.predict(X)

def generate_predictions_SKLsvm(λ=1.0, K="rbf", d=3, γ="scale", τ=-1, seed=None):
	start_time = time.time()
	model = train_SKLsvm(np.array(X_train), np.array(y_train), λ, K, d, γ, τ, seed)
	predictions = predict_SKLsvm(model, np.array(X_test))
	os.makedirs("./predictions/SKLsvm/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLsvm/", f"λ={λ},K={K},d={d},γ={γ},τ={τ},seed={seed}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### SKLgb Model

In [9]:
def train_SKLgb(X_train, y_train, α=0.1, η=100, ss=1.0, ψ=2, l=1, Δ=3, seed=None, θ=None):
	model = GradientBoostingClassifier(
		learning_rate=α,
		n_estimators=η,
		subsample=ss,
		min_samples_split=ψ,
		min_samples_leaf=l,
		max_depth=Δ,
		random_state=seed,
		max_leaf_nodes=θ
	)
	model.fit(X_train, y_train)
	return model

def predict_SKLgb(model, X): return model.predict(X)

def generate_predictions_SKLgb(α=0.1, η=100, ss=1.0, ψ=2, l=1, Δ=3, seed=None, θ=None):
	start_time = time.time()
	model = train_SKLgb(np.array(X_train), np.array(y_train), α, η, ss, ψ, l, Δ, seed, θ)
	predictions = predict_SKLgb(model, np.array(X_test))
	os.makedirs("./predictions/SKLgb/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLgb/", f"α={α},η={η},ss={ss},ψ={ψ},l={l},Δ={Δ},seed={seed},θ={θ}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### SKLhgb Model

In [10]:
def train_SKLhgb(X_train, y_train, α=0.1, τ=100, θ=31, Δ=None, l=20, seed=None):
	model = HistGradientBoostingClassifier(
		learning_rate=α,
		max_iter=τ,
		max_leaf_nodes=θ,
		max_depth=Δ,
		min_samples_leaf=l,
		random_state=seed
	)
	model.fit(X_train, y_train)
	return model

def predict_SKLhgb(model, X): return model.predict(X)

def generate_predictions_SKLhgb(α=0.1, τ=100, θ=31, Δ=None, l=20, seed=None):
	start_time = time.time()
	model = train_SKLhgb(np.array(X_train), np.array(y_train), α, τ, θ, Δ, l, seed)
	predictions = predict_SKLhgb(model, np.array(X_test))
	os.makedirs("./predictions/SKLhgb/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLhgb/", f"α={α},τ={τ},θ={θ},Δ={Δ},l={l},seed={seed}.csv")
	submission = pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### XGBgb Model

In [11]:
def train_XGBgb(X_train, y_train, η=100, Δ=3, θ=0, α=0.1, O="binary:logistic", B="gbtree", γ=0, ss=1, λ=0, Λ=1, seed=None):
	xgb_model = XGBClassifier(
		n_estimators=η,
		max_depth=Δ,
		max_leaves=θ if θ > 0 else None,
		learning_rate=α,
		objective=O,
		booster=B,
		gamma=γ,
		subsample=ss,
		reg_alpha=λ,
		reg_lambda=Λ,
		random_state=seed,
		n_jobs=-1
	)
	xgb_model.fit(X_train, y_train.ravel())
	return xgb_model

def predict_XGBgb(model, X):
	return model.predict(X)

def generate_predictions_XGBgb(η=100, Δ=3, θ=0, α=0.1, O="binary:logistic", B="gbtree", γ=0, ss=1, λ=0, Λ=1, seed=None):
	start_time = time.time()
	xgb_model = train_XGBgb(np.array(X_train), np.array(y_train), η, Δ, θ, α, O, B, γ, ss, λ, Λ, seed)
	predictions = predict_XGBgb(xgb_model, np.array(X_test))
	os.makedirs("./predictions/XGBgb/", exist_ok=True)
	file_name = os.path.join("./predictions/XGBgb/", f"η={η},Δ={Δ},θ={θ},α={α},O={O},B={B},γ={γ},ss={ss},λ={λ},Λ={Λ},seed={seed}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### CBgb Model

In [12]:
def train_CBgb(X_train, y_train, α=0.1, τ=100, Δ=6, l=3, seed=None):
	model = CatBoostClassifier(
		learning_rate=α,
		iterations=τ,
		depth=Δ,
		l2_leaf_reg=l,
		random_seed=seed
	)
	model.fit(X_train, y_train)
	return model

def predict_CBgb(model, X): return model.predict(X)

def generate_predictions_CBgb(α=0.1, τ=1, Δ=6, l=3, seed=None):
	start_time = time.time()
	model = train_CBgb(np.array(X_train), np.array(y_train), α, τ, Δ, l, seed)
	predictions = predict_CBgb(model, np.array(X_test))
	os.makedirs("./predictions/CBgb/", exist_ok=True)
	file_name = os.path.join("./predictions/CBgb/", f"α={α},τ={τ},Δ={Δ},l={l},seed={seed}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### Cross-Validation Area

In [13]:
def crossvalidate(train_fn, predict_fn, grid, X, y, k=2):
	kf = KFold(n_splits=k, shuffle=True)
	best_f1 = -np.inf
	best_hyperparameters = None

	for hyperparameters in grid:
		f1_scores = []
		for i_train, i_val in kf.split(X):
			X_train, X_val = X[i_train], X[i_val]
			y_train, y_val = y[i_train], y[i_val]
			model = train_fn(X_train, y_train, **hyperparameters)
			y_pred = predict_fn(model, X_val)
			f1_scores.append(f1_score(y_val, y_pred, average="macro"))
		mean_f1 = np.mean(f1_scores)
		if mean_f1 > best_f1:
			best_f1 = mean_f1
			best_hyperparameters = hyperparameters
	print(f"Best hyperparameters in grid:", best_hyperparameters)
	print(f"Best {k}-fold cross-validation f1 score:", best_f1)
	return best_hyperparameters, best_f1

def crossvalidate_and_generate_predictions(model_name, grid, k=5):
	train_fn = globals().get(f"train_{model_name.lower()}")
	predict_fn = globals().get(f"predict_{model_name.lower()}")
	generate_predictions_fn = globals().get(f"generate_predictions_{model_name.lower()}")
	if not train_fn or not predict_fn or not generate_predictions_fn:
		raise ValueError(f"Expected train_{model_name} or predict_{model_name} to exist.")
	best_hyperparameters, _ = crossvalidate(train_fn, predict_fn, grid, X_train, y_train, k=k)
	generate_predictions_fn(**best_hyperparameters)

In [14]:
# grid = [
# 	{"τ": 10000, "α": 0.0825, "G": "mini-batch", "β": 128, "R": "L2", "λ": 1},
# 	{"τ": 10000, "α": 0.085, "G": "mini-batch", "β": 128, "R": "L2", "λ": 1}
# ]
# crossvalidate_and_generate_predictions("logreg", grid, k=2)

# grid = [
# 	{"η": 100, "C": "gini", "Δ": None, "ψ": 2, "l": 1, "θ": None, "seed": None},
# 	{"η": 200, "C": "entropy", "Δ": 10, "ψ": 3, "l": 2, "θ": 50, "seed": 42}
# ]
# crossvalidate_and_generate_predictions("SKLrf", grid, k=5)

### Simulated Annealing Area

In [15]:
def accept_prob(old_score, new_score, T):
	T += 0.01
	return np.exp((new_score - old_score) / T)

def dt(t0, t1): return t1 - t0 if t0 is not None else 0

class CustomSimulatedAnnealing:
	def __init__(self, estimator, param_grid, scoring=f1_score, T=5, T_min=0.1, alpha=0.9, n_trans=5, max_iter=1, max_runtime=60, cv=2, verbose=True, refit=False, n_jobs=-1, max_score=np.inf):
		self.estimator = estimator
		self.param_grid = param_grid
		self.scoring = make_scorer(scoring)
		self.T = T
		self.T_min = T_min
		self.alpha = alpha
		self.n_trans = n_trans
		self.max_iter = max_iter
		self.max_runtime = max_runtime
		self.cv = cv
		self.verbose = verbose
		self.refit = refit
		self.n_jobs = n_jobs
		self.max_score = max_score

		self.best_params_ = None
		self.best_score_ = None
		self.best_estimator_ = None
		self.grid_scores_ = None
		self.runtime_ = None
		self._set_dynamic_params()

	def _set_dynamic_params(self):
		num_params = np.prod([len(v) for v in self.param_grid.values()])
		self.T = 10 * num_params
		self.T_min = 0.01 * self.T
		self.alpha = 0.9

	def fit(self, X, y):
		if isinstance(X, pd.DataFrame):
			X = X.to_numpy()
		if isinstance(y, pd.DataFrame):
			y = y.to_numpy()
		elif isinstance(y, (list, pd.Series)):
			y = np.array(y)

		T = self.T
		T_min = self.T_min
		alpha = self.alpha
		max_iter = self.max_iter
		n_trans = self.n_trans
		param_grid = self.param_grid
		max_runtime = self.max_runtime
		cv = self.cv

		old_params = {k: np.random.choice(v) for k, v in param_grid.items()}
		old_est = clone(self.estimator)
		old_est.set_params(**old_params)
		old_score, old_std = self._evaluate_score(old_est, X, y, cv)

		best_score = old_score
		best_params = old_params
		states_checked = {tuple(sorted(old_params.items())): (old_score, old_std)}
		total_iter = 1
		grid_scores = [(1, T, old_score, old_std, old_params)]

		time_at_start = time.time()
		t_elapsed = dt(time_at_start, time.time())

		while T > T_min and total_iter < max_iter and t_elapsed < max_runtime and best_score < self.max_score:
			for _ in range(self.n_trans):
				new_params = self._generate_new_params(old_params, param_grid)
				new_score, new_std = self._evaluate_score_for_params(new_params, X, y, cv, states_checked)

				if new_score >= self.max_score:
					break

				grid_scores.append((total_iter, T, new_score, new_std, new_params))

				if new_score > best_score:
					best_score = new_score
					best_params = new_params

				if self.verbose:
					print(f"{total_iter} T: {T:.5f}, score: {new_score:.6f}, std: {new_std:.6f}, params: {new_params}")

				if accept_prob(old_score, new_score, T) > random.random():
					old_params = new_params
					old_score = new_score

				t_elapsed = dt(time_at_start, time.time())
				total_iter += 1

			if new_score >= self.max_score:
				print(f"Max score reached {new_score}!")
				break

			T *= alpha

		if self.refit:
			self.estimator.set_params(**best_params)
			self.estimator.fit(X, y)
			self.best_estimator_ = self.estimator

		self.runtime_ = t_elapsed
		self.grid_scores_ = grid_scores
		self.best_score_ = best_score
		self.best_params_ = best_params

	def _generate_new_params(self, old_params, param_grid):
		new_params = old_params.copy()
		rand_key = np.random.choice(list(param_grid.keys()))
		val = param_grid[rand_key]
		if hasattr(val, 'rvs'):
			new_params[rand_key] = val.rvs()
		else:
			sample_space = [v for v in val if v != old_params[rand_key]]
			new_params[rand_key] = np.random.choice(sample_space) if sample_space else np.random.choice(val)
		return new_params

	def _evaluate_score_for_params(self, params, X, y, cv, states_checked):
		params_tuple = tuple(sorted(params.items()))
		if params_tuple in states_checked:
			return states_checked[params_tuple]
		else:
			est = clone(self.estimator)
			est.set_params(**params)
			score, std = self._evaluate_score(est, X, y, cv)
			states_checked[params_tuple] = (score, std)
			return score, std

	def _evaluate_score(self, estimator, X, y, cv):
		if self.n_jobs > 1:
			out = Parallel(n_jobs=self.n_jobs)(
				delayed(_fit_and_score)(clone(estimator), X, y, self.scoring, train, test, self.verbose,
										{}, {}, return_parameters=False, error_score='raise')
				for train, test in KFold(cv).split(X)
			)
		else:
			scores = []
			for train, test in KFold(cv).split(X):
				estimator.fit(X[train], y[train])
				scores.append(self.scoring(estimator, X[test], y[test]))
			out = (np.mean(scores), np.std(scores))
		return out

def simulated_annealing(model_name, param_grid, T=10, T_min=0.001, alpha=0.9, n_trans=5, max_iter=100, max_runtime=60, cv=5):
	param_grid_mapped = {PARAMETER_MAPPINGS.get(k, k): v for k, v in param_grid.items()}
	model_class = MODEL_CLASSES.get(model_name)
	if model_class is None: raise ValueError(f"Model '{model_name}' is not recognized.")
	model = model_class()
	sa = CustomSimulatedAnnealing(
		estimator=model,
		param_grid=param_grid_mapped,
		scoring=f1_score,
		T=T,
		T_min=T_min,
		alpha=alpha,
		n_trans=n_trans,
		max_iter=max_iter,
		max_runtime=max_runtime,
		cv=cv
	)
	sa.fit(X_train, y_train)
	return sa.best_estimator_, sa.best_score_, sa.best_params_, sa.grid_scores_, sa.runtime_

In [16]:
param_grid = {
	"α": [0.08, 0.1],
	"τ": [50],
	# "θ": [31, 50],
	# "Δ": [None, 10],
	# "l": [20]
}

best_model, best_score, best_params, grid_scores, runtime = simulated_annealing(
	model_name="SKLhgb",
	param_grid=param_grid
)

/Users/gregory/miniconda3/envs/Jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gregory/miniconda3/envs/Jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gregory/miniconda3/envs/Jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gregory/miniconda3/envs/Jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A 

1 T: 20.00000, score: 0.526579, std: 0.010613, params: {'learning_rate': 0.08, 'max_iter': 50}


/Users/gregory/miniconda3/envs/Jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gregory/miniconda3/envs/Jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gregory/miniconda3/envs/Jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gregory/miniconda3/envs/Jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A 

2 T: 20.00000, score: 0.539027, std: 0.010463, params: {'learning_rate': 0.1, 'max_iter': 50}
3 T: 20.00000, score: 0.526579, std: 0.010613, params: {'learning_rate': 0.08, 'max_iter': 50}
4 T: 20.00000, score: 0.539027, std: 0.010463, params: {'learning_rate': 0.1, 'max_iter': 50}
5 T: 20.00000, score: 0.539027, std: 0.010463, params: {'learning_rate': 0.1, 'max_iter': 50}
6 T: 18.00000, score: 0.526579, std: 0.010613, params: {'learning_rate': 0.08, 'max_iter': 50}
7 T: 18.00000, score: 0.526579, std: 0.010613, params: {'learning_rate': 0.08, 'max_iter': 50}
8 T: 18.00000, score: 0.539027, std: 0.010463, params: {'learning_rate': 0.1, 'max_iter': 50}
9 T: 18.00000, score: 0.526579, std: 0.010613, params: {'learning_rate': 0.08, 'max_iter': 50}
10 T: 18.00000, score: 0.526579, std: 0.010613, params: {'learning_rate': 0.08, 'max_iter': 50}
11 T: 16.20000, score: 0.539027, std: 0.010463, params: {'learning_rate': 0.1, 'max_iter': 50}
12 T: 16.20000, score: 0.526579, std: 0.010613, param

/Users/gregory/miniconda3/envs/Jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
